In [ ]:
import requests


In [ ]:
### get row from main table, where links should be set
def nocodb_get_rows_from_table(table_id, view_id):
    url = f'{noco_base_url}/api/v2/tables/{table_id}/records?viewId={view_id}&limit=5000'
    headers = {
        'accept': 'application/json',
        'xc-token': xc_token,
    }

    print(url)
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if 'list' in data:
            return data['list']
        else:
            print("No 'list' key found in the response.")
            return None
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return None

rows = nocodb_get_rows_from_table(table_id, view_id)
print(rows)


In [ ]:
## get the primary key from the found rows

ids_in_table={}

def nocodb_get_id_from_rows():
    if rows:
        for row in rows:
            #ids_in_table.append(row['Id'])
            ids_in_table[row['Id']]=(row[fieldname])
        return ids_in_table

ids_in_table = nocodb_get_id_from_rows()
print(ids_in_table)

for row_id in ids_in_table:
    print("------------------------------------------------------------------")
    # get object_id from sub_table based on row id from main table
    primary_key_main = row_id
    print(f"primary_key_main: {primary_key_main}")
    object_id=ids_in_table.get(row_id)
    print(f"object_id: {object_id}")

In [ ]:
from urllib.parse import quote

def nocodb_get_column_from_rows_by_primary_key(sub_table_id, fieldname, object_id):

    #if row_id is None:
    #    print("No row ID found in the table.")
    #    return column_data

    #url = f'http://localhost:8085/api/v2/tables/{sub_table_id}/records/{row_id}?fields={column}'
    url = f'{noco_base_url}/api/v2/tables/{sub_table_id}/records?fields=Id%2C{fieldname}&where=where%3D%28{fieldname}%2Ceq%2C{object_id}%29'
    #url = f'https://tables.rhizome.org/api/v2/tables/mh3vo19pcc4px5a/records/{object_id}?fields=Id%2C{fieldname}'
    url_encoded = quote(url)
    
    print(object_id)
    print(url)
    print(url_encoded)
    
    headers = {
        'accept': 'application/json',
        'xc-token': xc_token,
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        print(data)
        id_values = []
        if 'list' in data and data['list']:
            # Accessing the ID
            id_values = [item['Id'] for item in data['list'] if 'Id' in item]
            
            if id_values is not None:
                print("ID:", id_values)
            else:
                print("ID not found.")
        else:
            print("The list is empty or does not exist in the data.")
        #print(f"data: {data}")
        #print(f"id_values: {id_values}")
        #column_data[row_id]=(data[column])
        return id_values
    else:
        print(f"Failed to fetch data for {fieldname} {3974}. Status code: {response.status_code}")

id_sub_table_list = nocodb_get_column_from_rows_by_primary_key(sub_table_id, fieldname, object_id)


In [ ]:
def nocodb_set_link_by_id(table_id, link_field_id, view_id):

    for row_id in ids_in_table:
        print("------------------------------------------------------------------")
        # get object_id from sub_table based on row id from main table
        primary_key_main = row_id
        print(f"primary_key_main: {primary_key_main}")
        object_id=ids_in_table.get(row_id)
        print(f"object_id: {object_id}")

        if isinstance(id_sub_table_list, list):
            for id_sub_table in id_sub_table_list:
                print(f"primary_key_sub_table: {id_sub_table}")
                url = f'{noco_base_url}/api/v2/tables/{table_id}/links/{link_field_id}/records/{row_id}'
                headers = {
                    'accept': 'application/json',
                    #'xc-auth': xc_auth,
                    'xc-token': xc_token,
                    'Content-Type': 'application/json'
                }
                data = {
                    "id": id_sub_table
                }

                response = requests.post(url, headers=headers, json=data)
                print(response.text)
        else:
            print(f"No IDs found for object_id: {object_id}")

#table_id="mbe6r50mzekxwap"
response = nocodb_set_link_by_id(table_id, link_field_id, view_id)
print(response)